# Extractions

The objective of this notebook is to generate extractions from the verbatims.

In [1]:
import pandas as pd
import json

from tqdm import tqdm
from typing import List, Dict
from pymongo import MongoClient

import boto3
import certifi

from utils.extractions_utils import generate_extractions, split_text_parts, get_extractions, add_extractions_to_splitted_analysis

In [2]:
STAGE = 'prod'

Loading all key libraries

In [3]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"{STAGE.capitalize()}/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)

OPENAI_API_KEY = _secrets["openai"]["api_key"]
LLM_API_KEY = _secrets["litellm"]["api_key"]


In [4]:
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

Define the brand and a short description of the brand.

Here `columbuscafe_test` is a brand for testing purposes.

In [5]:
BRAND = 'columbuscafe_test'

BRAND_DESCR = '''
Feedbacks are from client of columbus cafe.
Columbus Café & Co est une chaîne française de cafés.
'''

In [6]:
from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))

from_mongo.shape

(2994, 16)

Displaying a random verbatim to understand the data.

In [7]:
row = from_mongo.sample().iloc[0]
text = row.verbatim['text']

print(text)

Bon café lacté, service rapide et prix correct.


First we will split our text into different parts by using punctuation.

In [8]:
text_parts = split_text_into_parts(text)

text_parts

[{'text': 'Bon café lacté,'}, {'text': ' service rapide et prix correct.'}]

Then this is an example of generating extractions from the text parts.

In [9]:
result = extract_information_from_text(
    text=text,
    id=row['id'],
    brand_descr=BRAND_DESCR,
    language='french',
    model='gpt-4o-mini'
)

extractions = result.get('extraction')

splitted_analysis = result.get('splitted_analysis')

In [10]:
extractions

[{'sentiment': 'POSITIVE',
  'extraction': 'Café lacté apprécié',
  'text': 'Bon café lacté,'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Service rapide',
  'text': ' service rapide et prix correct.'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Prix correct',
  'text': ' service rapide et prix correct.'}]

In  `splitted_analysis` we reassemble the extractions with the corresponding text parts

In [11]:
splitted_analysis

[{'text': 'Bon café lacté,',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Café lacté apprécié'}]},
 {'text': ' service rapide et prix correct.',
  'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Service rapide'},
   {'sentiment': 'POSITIVE', 'extraction': 'Prix correct'}]}]

## Run the extraction pipeline

In [12]:
# this function allows to parallelize the extraction process and to save the results on the mongo database
from utils.extractions_utils import process_extractions_in_parallel

In [13]:
subdf = from_mongo.sample(20) # test on a subset
print('To extract:', subdf.shape[0])

subdf['text'] = subdf.verbatim.apply(lambda x: x['text'] if isinstance(x, dict) and 'text' in x else '',)
texts_with_ids = subdf[['text', '_id']].to_dict(orient='records')

To extract: 20


In [14]:
extractions = process_extractions_in_parallel(
    texts_with_ids,
    brand_name=BRAND,
    brand_descr=BRAND_DESCR,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=True
)

Processing chunks:   0%|          | 0/1 [00:00<?, ?it/s]

[parse_extraction()] Il semble que vous n'ayez pas fourni de texte à analyser. Veuillez soumettre le texte contenant les retours des clients de Columbus Café & Co, et je pourrai alors extraire les informations pertinentes comme demandé. invalid syntax (<string>, line 0)
 columbuscafe_test/2918a01a29f744cec7a3
[get_extractions()] 'text'
[parse_extraction()] Il semble que vous n'ayez pas fourni de texte à analyser. Veuillez soumettre le texte contenant les retours des clients de Columbus Café & Co, et je pourrai alors extraire les informations pertinentes comme demandé. invalid syntax (<string>, line 0)
 columbuscafe_test/1976c554cdd362249798
[get_extractions()] 'text'


Processing chunks: 100%|██████████| 1/1 [00:10<00:00, 10.14s/it]


In [16]:
extractions

[{'id': 'columbuscafe_test/f6790519b4ff3fd7b410',
  'splitted_analysis': [{'text': 'Je viens régulièrement prendre mon petit moka au colombus et c’est toujours un plaisir d’être reçu par Iris qui est souriante et serviable !',
    'extractions': [{'sentiment': 'POSITIVE',
      'extraction': "Satisfaction de l'accueil par Iris"}]},
   {'text': ' De plus,'},
   {'text': ' son moka est exceptionnel 🤩',
    'extractions': [{'sentiment': 'POSITIVE',
      'extraction': 'Moka exceptionnel'}]}],
  'extraction': [{'sentiment': 'POSITIVE',
    'extraction': "Satisfaction de l'accueil par Iris",
    'text': 'Je viens régulièrement prendre mon petit moka au colombus et c’est toujours un plaisir d’être reçu par Iris qui est souriante et serviable !'},
   {'sentiment': 'POSITIVE',
    'extraction': 'Moka exceptionnel',
    'text': ' son moka est exceptionnel 🤩'}]},
 {'id': 'columbuscafe_test/a965ce53ffc4263dd876',
  'splitted_analysis': [{'text': 'Très propre le personnel est très aimable je vous 

## Checking result in MongoDB

In [16]:
result

{'id': '0076709ae318c34cd7ff',
 'splitted_analysis': [{'text': 'Bon café lacté,',
   'extractions': [{'sentiment': 'POSITIVE',
     'extraction': 'Café lacté apprécié'}]},
  {'text': ' service rapide et prix correct.',
   'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Service rapide'},
    {'sentiment': 'POSITIVE', 'extraction': 'Prix correct'}]}],
 'extraction': [{'sentiment': 'POSITIVE',
   'extraction': 'Café lacté apprécié',
   'text': 'Bon café lacté,'},
  {'sentiment': 'POSITIVE',
   'extraction': 'Service rapide',
   'text': ' service rapide et prix correct.'},
  {'sentiment': 'POSITIVE',
   'extraction': 'Prix correct',
   'text': ' service rapide et prix correct.'}]}

In [17]:
# getting the document from the database to check if the extractions are saved
# matching the brand and the id in res

documents = collection.find({
    'brand': BRAND,
    '_id': {'$in': [r['id'] for r in extractions]}
})

documents = pd.DataFrame(documents)

In [18]:
documents

,_id,id,brand,timestamp,verbatim,establishment,review_site,author,rating_out_of_5,language,extractions,splitted_analysis_v2,review_title
0,columbuscafe_test/0c62fc38458d9ae2a535,0c62fc38458d9ae2a535,columbuscafe_test,1.684109e+12,{'text': 'Belle décoration mais très mauvais s...,Romans Marques Avenue,Google,Kai Z,1.0,fr,"[{'sentiment': 'NEGATIVE', 'extraction': 'Mauv...",[{'text': 'Belle décoration mais très mauvais ...,NaN
1,columbuscafe_test/12e3c568ad741bf2ef96,12e3c568ad741bf2ef96,columbuscafe_test,1.692058e+12,{'text': 'Attendu 25 mn pour un café …. 1 serv...,Saint-Laurent-du-Var,Google,Enzorun Enzo,1.0,fr,"[{'sentiment': 'NEGATIVE', 'extraction': 'Atte...","[{'text': 'Attendu 25 mn pour un café ….', 'ex...",NaN
2,columbuscafe_test/1379d41c2db53db619b0,1379d41c2db53db619b0,columbuscafe_test,1.693699e+12,{'text': '2👍'},Caen Rives de l’Orne,Uber Eats,Alice C,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Appr...","[{'text': '2👍', 'extractions': [{'sentiment': ...",01-09-2023 - 12.1EUR
3,columbuscafe_test/169c5abb63e57319c09d,169c5abb63e57319c09d,columbuscafe_test,1.697155e+12,{'text': 'Toujours aussi bien tenu. Déco au t...,Fnac Bordeaux - Bordeaux Sainte-Catherine,Google,Naoual EL HIMASS,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Étab...","[{'text': 'Toujours aussi bien tenu.', 'extrac...",NaN
4,columbuscafe_test/189aca20a5a4b115a976,189aca20a5a4b115a976,columbuscafe_test,1.683072e+12,{'text': 'Good coffee'},Béziers Allée Paul Riquet,Google,Philip Painter,4.0,en,"[{'sentiment': 'POSITIVE', 'extraction': 'Bon ...","[{'text': 'Good coffee', 'extractions': [{'sen...",NaN
5,columbuscafe_test/1976c554cdd362249798,1976c554cdd362249798,columbuscafe_test,1.690848e+12,{},Dijon Godrans,Google,Robin Oser,4.0,fr,[],[],NaN
6,columbuscafe_test/1d0e491e1394918bfc35,1d0e491e1394918bfc35,columbuscafe_test,1.692058e+12,{'text': 'Super accueillant'},Bourgoin-Jallieu,Google,pascal m,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Accu...","[{'text': 'Super accueillant', 'extractions': ...",NaN
7,columbuscafe_test/2918a01a29f744cec7a3,2918a01a29f744cec7a3,columbuscafe_test,1.696378e+12,{},Le Plessis-Robinson,Google,rafal drypa,5.0,fr,[],[],NaN
8,columbuscafe_test/2bf8d9e246f803f8808d,2bf8d9e246f803f8808d,columbuscafe_test,1.695341e+12,"{'text': 'Endroit sympa, big up aux pains au c...",Tours Change,Google,Emeline L,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Endr...","[{'text': 'Endroit sympa,', 'extractions': [{'...",NaN
9,columbuscafe_test/3637b6c505ac86f55a53,3637b6c505ac86f55a53,columbuscafe_test,1.688602e+12,"{'text': 'Très bon, et très serviable de leur ...",Décathlon Annemasse,Google,tatakay,5.0,fr,"[{'sentiment': 'POSITIVE', 'extraction': 'Serv...","[{'text': 'Très bon,', 'extractions': [{'senti...",NaN


In [26]:
documents.sample().iloc[0].extractions

[{'sentiment': 'POSITIVE', 'extraction': 'Bon café', 'text': 'Good coffee'}]

In [31]:
documents.sample().iloc[0].splitted_analysis_v2

[{'text': 'Bonjour,'},
 {'text': " pourquoi ca ferme à 19h n'est-ce pas étrange comme horaire ?",
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Heure de fermeture à 19h'}]},
 {'text': ' Bizarre comme modele non ?',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Modèle de fonctionnement étrange'}]}]